In [ ]:
!pip install librosa
!pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 89.3 MB/s eta 0:00:00
  Created wheel for yt-dlp: filename=yt_dlp-2023.3.4-py2.py3-none-any.whl size=2750017 sha256=d0a69695adc44e83b221fddaee0d369c65d2f9a5b4f77cce02

In [ ]:
import librosa
import numpy as np
from __future__ import unicode_literals
import yt_dlp as youtube_dl
import soundfile as sf
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Olga
def downloadSongFromYT(url, outPath):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': outPath,
        'noplaylist': True,
        'continue_dl': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192', }]
    }
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.cache.remove()
            ydl.download([url])
            return True
    except Exception as e:
        print(e)
        return False 


def cutAudio(path, outPath, start, end): 
  y, sr = librosa.load(path)
  start_samples = int(start*sr)
  end_samples = int(end*sr)
  y_cut = y[start_samples : end_samples]
  
  sf.write(outPath, y_cut,  sr)
  return y_cut

In [ ]:
# -------------------------------------------------------------------------------------------NOT USED
#sr: framerate
#fact: changes the way the volumes are normalized 
#if audio is cut, simply provide the path to the cut 
def get_onsets_2d(path, sr, norm): 


  y, sr = librosa.load(path, sr=sr)
  
  y_n = (y-y.min())/(y.max()-y.min())*(1) + 1
  max = y_n.max()
  #print(y_n)
  #print('Max', y_n.max())
  #print('Min', y_n.min())
  #print('Len', len(y_n))
  
  for i in range(len(y_n)): 
    if(y_n[i] < max-norm): 
      y_n[i] = 1.0

  peaks = np.zeros(len(y_n))
  for i in range(len(peaks)): 
    if(y_n[i] != 1.0): 
      peaks[i] = 1
  #print('peaks', peaks)
  #0.5 delta massimo
  #fact, decide il range
  sd_list  = ""
  sd_list_n = ""
  for i in range(len(y_n)) :
    
    if(i != len(y_n) -1  ): 
      sd_list = sd_list + str(i) + ":(" + str(y_n[i]) + "), "
    else: 
      sd_list = sd_list + str(i) + ":(" + str(y_n[i]) + ")"
  
  #print('before\n', sd_list)
  for i in range(len(y_n)) :
    
    if(i != len(y_n) -1  ): 
      if(peaks[i+1] == 1.0 and i < len(y_n) - 2 ): 
   #     print('check',i, y_n[i+1]-0.1 )
        sd_list_n = sd_list_n + str(i) + ":(" + str(y_n[i+1]-0.1) + "), "

      else:
        sd_list_n = sd_list_n + str(i) + ":(" + str(y_n[i]) + "), "

    else: 
      sd_list_n = sd_list_n + str(i) + ":(" + str(y_n[i]) + ")"
    
  
  #print('after\n', sd_list_n)
  
  return sd_list_n



  #print(sd_list) 
#https://www.youtube.com/watch?v=EqTZsJMBZw0



In [ ]:
# -------------------------------------------------------------------------------------------NOT USED
def get_onsets_3d(path, sr, norm): 


  y, sr = librosa.load(path, sr=sr)
  
  y_n = (y-y.min())/(y.max()-y.min())*(10)
  max = y_n.max()
  #print(y_n)
  #print('Max', y_n.max())
  #print('Min', y_n.min())
  #print('Len', len(y_n))
  
  for i in range(len(y_n)): 
    if(y_n[i] < max-norm): 
      y_n[i] = 0

  peaks = np.zeros(len(y_n))
  for i in range(len(peaks)): 
    if(y_n[i] != 0): 
      peaks[i] = 1
  #print('peaks', peaks)
  #0.5 delta massimo
  #fact, decide il range
  sd_list  = ""
  sd_list_n = ""
  for i in range(len(y_n)) :
    
    if(i != len(y_n) -1  ): 
      sd_list = sd_list + str(i) + ":(" + str(y_n[i]) + "), "
    else: 
      sd_list = sd_list + str(i) + ":(" + str(y_n[i]) + ")"
  
  #print('before\n', sd_list)
  for i in range(len(y_n)) :
    
    if(i != len(y_n) -1  ): 
      if(peaks[i+1] == 1.0 and i < len(y_n) - 2 ): 
   #     print('check',i, y_n[i+1]-0.1 )
        sd_list_n = sd_list_n + str(i) + ":(" + str(y_n[i+1]-2) + "), "

      else:
        sd_list_n = sd_list_n + str(i) + ":(" + str(y_n[i]) + "), "

    else: 
      sd_list_n = sd_list_n + str(i) + ":(" + str(y_n[i]) + ")"
    
  
  #print('after\n', sd_list_n)
  
  return sd_list_n

In [ ]:
# -------------------------------------------------------------------------------------------NOT USED
def get_onsets_angles(path, sr, norm, delta): 


  y, sr = librosa.load(path, sr=sr)
  
  y_n = (y-y.min())/(y.max()-y.min())*(-2*delta) + delta
  max = abs(y_n).max()
  #print(y_n)
  #print('Max', y_n.max())
  #print('Min', y_n.min())
  #print('Len', len(y_n))
  
  for i in range(len(y_n)): 
    if(abs(y_n[i]) < max-norm ): 
      y_n[i] = 0

  peaks = np.zeros(len(y_n))
  for i in range(len(peaks)): 
    if(y_n[i] != 0): 
      peaks[i] = 1
  #print('peaks', peaks)
  #0.5 delta massimo
  #fact, decide il range
  sd_list  = ""
  sd_list_n = ""
  for i in range(len(y_n)) :
    
    if(i != len(y_n) -1  ): 
      sd_list = sd_list + str(i) + ":(" + str(y_n[i]) + "), "
    else: 
      sd_list = sd_list + str(i) + ":(" + str(y_n[i]) + ")"
  
  #print('before\n', sd_list)
  for i in range(len(y_n)) :
    
    if(i != len(y_n) -1  ): 
      if(peaks[i+1] == 1.0 and i < len(y_n) - 2 ): 
   #     print('check',i, y_n[i+1]-0.1 )
        sd_list_n = sd_list_n + str(i) + ":(" + str(y_n[i+1]-2) + "), "

      else:
        sd_list_n = sd_list_n + str(i) + ":(" + str(y_n[i]) + "), "

    else: 
      sd_list_n = sd_list_n + str(i) + ":(" + str(y_n[i]) + ")"
    
  
  #print('after\n', sd_list_n)
  
  return sd_list_n

In [ ]:
def get_beats_librosa_zoom(path, sr, tempo, ts): 
  y, sr_x = librosa.load(path)
  
  #onset_envelope = librosa.onset.onset_strength(y=y, sr=sr_x)
  
  #onsets = librosa.onset.onset_detect(onset_envelope=onset_envelope)
  tempo, beats =  librosa.beat.beat_track(y=y, sr=sr_x, bpm = tempo)
  beats_times = librosa.frames_to_time(beats)
  y_d, sr_d = librosa.load(path, sr = sr)
  ts_fs = 1/sr_d
  
  array_fs = np.arange(0, len(y_d)*ts_fs, ts_fs)
  #print('ar', array_fs)
  new_beats = []
  tc = 0 
  for i in range(len(array_fs)): 

    if(i<len(array_fs) -2 ): 

      while((beats_times[tc] > array_fs[i] and beats_times[tc] < array_fs[i+1]) and (tc < beats_times.size - 1 )): 
        if((tc+1)%ts==0):
          new_beats.append(i)
          #print('bt', beats_times[tc])
        #same values may be present
        #controllo su length di onset times !!!! 
          #if((beats_times[tc] > array_fs[i] + ts_fs/2) and (beats_times[tc] < array_fs[i] + ts_fs)): 
          #  new_beats.append(i)
          #elif((beats_times[tc] > array_fs[i]) and (beats_times[tc] < array_fs[i] + ts_fs/2)): 
          #  new_beats.append(i+1)
        tc = tc +1

  #print(new_beats)
  sd_list_n = ""
  beats_ar = np.full(len(array_fs), 1.0)



  for i in range(len(beats_ar)): 
    for b in new_beats:
      if(i == b): 
        beats_ar[i] = random.uniform(1.0, 2.0)

#Scrivi funzione tipo 1.0 1.0 1.0  besa
    #print('before\n', sd_list)
  for i in range(len(beats_ar)) :
 

    if(i != len(beats_ar) -1  ): 
      
        if(beats_ar[i+1] != 1.0 ): 
          sd_list_n = sd_list_n + str(i) + ":(" + str(beats_ar[i+1]-0.1) +"), "
        else: 
          sd_list_n = sd_list_n + str(i) + ":("+ str(beats_ar[i]) +"), "
    else: 
       sd_list_n = sd_list_n + str(i) + ":("+ str(beats_ar[i]) +")"
    
   #   if(i != len(new_beats) -1  ): 
   #     sd_list_n = sd_list_n + str(new_beats[i]) + ":("+ str(val) +"), "
   #   else: 
   #     sd_list_n = sd_list_n + str(new_beats[i]) + ":("+ str(val) +")"


  return sd_list_n



In [ ]:
def get_beats_librosa_angle(path, sr, tempo, ts): 
  y, sr_x = librosa.load(path)
  
  #onset_envelope = librosa.onset.onset_strength(y=y, sr=sr_x)
  
  #onsets = librosa.onset.onset_detect(onset_envelope=onset_envelope)
  tempo, beats =  librosa.beat.beat_track(y=y, sr=sr_x, bpm = tempo)
  beats_times = librosa.frames_to_time(beats)
  
  y_d, sr_d = librosa.load(path, sr = sr)
  
  ts_fs = 1/sr_d
  
  array_fs = np.arange(0, len(y_d)*ts_fs, ts_fs)
  #print('ar', array_fs)
  new_beats = []
  tc = 0 
  #print(beats_times)
  for i in range(len(array_fs)): 

    if(i<len(array_fs) -2 ): 

      while((beats_times[tc] > array_fs[i] and beats_times[tc] < array_fs[i+1]) and (tc < beats_times.size - 1 )): 
        #same values may be present
        #controllo su length di onset times !!!! 
        if((tc+1)%ts==0):
            new_beats.append(i)
          #if((beats_times[tc] > array_fs[i] + ts_fs/2) and (beats_times[tc] < array_fs[i] + ts_fs)): 
          #  new_beats.append(i)
          #elif((beats_times[tc] > array_fs[i]) and (beats_times[tc] < array_fs[i] + ts_fs/2)): 
          #  new_beats.append(i+1)
        tc = tc +1
  #print(new_beats)
  sd_list_n = ""
  beats_ar = np.full(len(array_fs), 0)



  for i in range(len(beats_ar)): 
    for b in new_beats:
      if(i == b): 
        beats_ar[i] = random.randint(-30, 30)

#Scrivi funzione tipo 1.0 1.0 1.0  besa
    #print('before\n', sd_list)
  for i in range(len(beats_ar)) :
 

    if(i != len(beats_ar) -1  ): 
      
       sd_list_n = sd_list_n + str(i) + ":("+ str(beats_ar[i]) +"), "
    else: 
       sd_list_n = sd_list_n + str(i) + ":("+ str(beats_ar[i]) +")"
    
   #   if(i != len(new_beats) -1  ): 
   #     sd_list_n = sd_list_n + str(new_beats[i]) + ":("+ str(val) +"), "
   #   else: 
   #     sd_list_n = sd_list_n + str(new_beats[i]) + ":("+ str(val) +")"


  return sd_list_n



In [ ]:
#angles = get_beats_librosa('/content/drive/MyDrive/out_cut.wav', 10, "ANGLE", 30)
#print('Angles', angles)
#zooms = get_beats_librosa_zoom('/content/drive/MyDrive/out_cut.wav', 10,  120, 4)
#angles = get_beats_librosa_angle('/content/drive/MyDrive/out_cut.wav', 10, 120, 4)
#print('zooms', zooms)
#print('angles', angles)


ar [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7
 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5
 3.6 3.7 3.8 3.9 4.  4.1 4.2 4.3 4.4 4.5 4.6 4.7 4.8 4.9 5.  5.1 5.2 5.3
 5.4 5.5 5.6 5.7 5.8 5.9 6.  6.1 6.2 6.3 6.4 6.5 6.6 6.7 6.8 6.9 7.  7.1
 7.2 7.3 7.4 7.5 7.6 7.7 7.8 7.9 8.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8 8.9
 9.  9.1 9.2 9.3 9.4 9.5 9.6 9.7 9.8 9.9]
bt 2.2291156462585033
bt 4.551111111111111
bt 6.896326530612245
[23, 45, 68]
zooms 0:(1.0), 1:(1.0), 2:(1.0), 3:(1.0), 4:(1.0), 5:(1.0), 6:(1.0), 7:(1.0), 8:(1.0), 9:(1.0), 10:(1.0), 11:(1.0), 12:(1.0), 13:(1.0), 14:(1.0), 15:(1.0), 16:(1.0), 17:(1.0), 18:(1.0), 19:(1.0), 20:(1.0), 21:(1.0), 22:(1.5960539887590794), 23:(1.6960539887590795), 24:(1.0), 25:(1.0), 26:(1.0), 27:(1.0), 28:(1.0), 29:(1.0), 30:(1.0), 31:(1.0), 32:(1.0), 33:(1.0), 34:(1.0), 35:(1.0), 36:(1.0), 37:(1.0), 38:(1.0), 39:(1.0), 40:(1.0), 41:(1.0), 42:(1.0), 43:(1.0), 44:(1.1028453225012347), 45:(1.2028453225012348